In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

In [12]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [13]:
DATA_PATH = "C:/Users/Ashwin/Projects/Image-Steganography/MLStego"
trainset = ImageFolder(DATA_PATH+"/train/", transform=transform)
testset = ImageFolder(DATA_PATH+"/test/", transform=transform)


In [14]:
trainset.classes

['Normal', 'Stego']

In [15]:
trainLoader=DataLoader(trainset,batch_size=32,shuffle=True)

In [16]:
class StegNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1=nn.Sequential(
            nn.Conv2d(3, 6, 3),
            nn.MaxPool2d(2, 2),
            # nn.BatchNorm2d(16)
        )
        self.relu=nn.ReLU()
        self.fc=nn.Linear(5766,2)
        self.softmax=nn.Softmax(dim=1)

    def forward(self,x):
        x=self.block1(x)
        x=self.relu(x)
        x=torch.flatten(x,1)
        x=self.fc(x)
        return x


In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net=StegNet().to(device)
optimizer=torch.optim.Adam(net.parameters())
criterion=nn.CrossEntropyLoss()


In [22]:
# loop over the trainset multiple times

correct = 0
for epoch in range(5):
    running_loss = 0.0

    total = 0
    for data in trainLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # labels=labels.reshape(-1,1)
        # outputs=outputs.float()
        # labels=labels.float()
        correct += (torch.argmax(outputs, 1) == labels).sum()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Loss: {} '.format(running_loss))
print(f"Accuracy: {correct/len(trainset)}")
print('Finished Training')


Loss: 18.09787678718567 
Loss: 18.051266133785248 
Loss: 18.02214205265045 
Loss: 18.019690215587616 
Loss: 18.022288501262665 
Accuracy: 2.532418966293335
Finished Training


In [19]:
classes = trainset.classes
testLoader = DataLoader(testset, batch_size=32, shuffle=True)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images,labels=images.to(device),labels.to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))

Accuracy for class Normal is: 0.0 %
Accuracy for class Stego is: 100.0 %


In [20]:
correct, total = 0, 0
for classname in correct_pred:
    correct += correct_pred[classname]
    total += total_pred[classname]

print("Test Accuracy: ", correct/total*100)

Test Accuracy:  50.0


In [21]:
correct_pred

{'Normal': 0, 'Stego': 99}